In [0]:
# Define os parâmetros da Job
dbutils.widgets.text('p_path','')
dbutils.widgets.text('p_layer','')
dbutils.widgets.text('p_file_name_bronze','')

dbutils.widgets.text('p_file_name_silver','')
dbutils.widgets.text('p_mode_write_silver','')
dbutils.widgets.text('p_modo_write_delta','')

In [0]:
# Importando Bibliotecas
from pyspark.sql import DataFrame, Row
from pyspark.sql.functions import current_timestamp, col
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, NumericType, DateType, BooleanType, TimestampType
from datetime import datetime

#### Funções para manipular arquivos parquet

In [0]:
def ler_parquet(file_path: str) -> DataFrame:
    """
    Lê um arquivo Parquet no Spark.

    Args:
        file_path (str): Caminho do arquivo Parquet.

    Returns:
        DataFrame: DataFrame resultante da leitura do arquivo Parquet.
    """
    try:
        # Lê o arquivo Parquet
        df = spark.read.parquet(file_path)
        print("Leitura do arquivo Parquet bem-sucedida.")
        return df
    except Exception as e:
        print(f"Erro ao ler o arquivo Parquet: {e}")
        return None

In [0]:
def gravar_parquet(df :DataFrame, destination_path :str, mode_write_bronze :str) -> None:
    """
    Grava os dados de um DataFrame no formato Parquet.

    Args:
        df (DataFrame): DataFrame do Spark.
        destination_path (str): Caminho de destino para salvar o arquivo Parquet.
    """
    # Grava o DataFrame no formato Parquet
    df.write.mode(mode_write_bronze).parquet(destination_path)

    print("Dados salvos com sucesso no formato Parquet.")


#### Função para tratamento de dados duplicados

In [0]:
def verificar_e_remover_duplicatas(df: DataFrame) -> DataFrame:
    """
    Verifica se há linhas duplicadas em um DataFrame do Spark e remove as duplicatas.

    Args:
    dataframe (DataFrame): DataFrame do Spark a ser processado.

    Returns:
    DataFrame: DataFrame resultante após a remoção das linhas duplicadas, ou None se ocorrer um erro.
    """

    # Verifica se o argumento é um DataFrame do Spark
    if not isinstance(df, DataFrame):
        print('Erro: O argumento não é um DataFrame do Spark.')

    try:
        # Verifica se há linhas duplicadas
        num_duplicatas = df.count() - df.dropDuplicates().count()

        # Se houver duplicatas, remove-as
        if num_duplicatas > 0:
            print('Existem linhas duplicadas. Removendo...')
            df_no_duplicates = df.dropDuplicates()
            return df_no_duplicates
        else:
            print('NÃO existem linhas duplicadas')
            return df

    except Exception as e:
        print(f"Erro ao verificar duplicatas: {e}")
        return None

In [0]:
# Define o esquema do DataFrame de exemplo
schema = StructType([
    StructField("id_pais", IntegerType(), True),
    StructField("nome_pais", StringType(), True)
])

# Dados do DataFrame de exemplo
data = [
    [1,'Brasil'],
    [2,'Argentina'],
    [3,'Chile'],
    [4,'Peru'],
    [1,'Brasil']
]

# Cria o DataFrame
df_test_duplicates = spark.createDataFrame(data, schema)

# Exibe o DataFrame
display(df_test_duplicates)

# Chamar a função para tratamento de valores duplicatas no DataFrame
df_trat_duplicates = verificar_e_remover_duplicatas(df_test_duplicates)

display(df_trat_duplicates)

id_pais,nome_pais
1,Brasil
2,Argentina
3,Chile
4,Peru
1,Brasil


Existem linhas duplicadas. Removendo...


id_pais,nome_pais
1,Brasil
2,Argentina
3,Chile
4,Peru


#### Função para tratamento de dados com valores nulos

In [0]:
def verificar_e_tratar_valores_nulos(df :DataFrame) -> DataFrame:
    """
    Verifica e trata os valores nulos em um DataFrame do Spark.

    Args:
    df (DataFrame): DataFrame do Spark a ser processado.

    Returns:
    DataFrame: DataFrame resultante após o tratamento dos valores nulos, ou None se ocorrer um erro.
    Tipos de colunas e seu tratamento:
      - Numérico: 0
      - Não numérico: 'Não informado'
      - Data: '1900-01-01'
      - Data e hora: '1900-01-01 00:00:00'
      - Booleano: False
    """
    try:
        # Conta a quantidade de valores nulos por coluna
        valores_nulos_por_coluna = {}
        for coluna in df.columns:
            num_nulos_na_coluna = df.where(df[coluna].isNull()).count()
            valores_nulos_por_coluna[coluna] = num_nulos_na_coluna

        # Mapeia cada coluna para seu valor padrão correspondente
        valores_padrao = {}
        for coluna in df.columns:
            tipo_coluna = df.schema[coluna].dataType

            if isinstance(tipo_coluna, IntegerType):
                valores_padrao[coluna] = 0
            elif isinstance(tipo_coluna, StringType):
                valores_padrao[coluna] = "Não informado"
            elif isinstance(tipo_coluna, DateType):
                valores_padrao[coluna] = "1900-01-01"
            elif isinstance(tipo_coluna, TimestampType):
                valores_padrao[coluna] = "1900-01-01 00:00:00"
            elif isinstance(tipo_coluna, BooleanType):
                valores_padrao[coluna] = False

        # Preenche os valores nulos em todas as colunas com os valores padrão
        df = df.fillna(valores_padrao)

        print('Valores nulos tratados por coluna:')
        for coluna, num_nulos_tratados in valores_nulos_por_coluna.items():
            print(f'{coluna}: {num_nulos_tratados}')

        print('Todos os valores nulos foram tratados.')

        return df

    except Exception as e:
        print(f'Erro ao verificar e tratar valores nulos: {e}')
        return None


In [0]:
# Define o esquema do DataFrame de exemplo
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("nome", StringType(), True),
    StructField("idade", IntegerType(), True),
    StructField("genero", StringType(), True),
    StructField("ativo", BooleanType(), True),
    StructField("data_nascimento", DateType(), True),
    StructField("ultima_atualizacao", TimestampType(), True)
])

# Dados do DataFrame de exemplo
dados = [
    (1, "João", 30, "Masculino", True, datetime.strptime("1992-05-15", "%Y-%m-%d").date(), datetime.now()),
    (2, "Maria", None, "Feminino", False, None, datetime.now()),
    (3, None, 25, None, True, datetime.strptime("1997-10-20", "%Y-%m-%d").date(), None)
]


# Cria o DataFrame
df_test_null = spark.createDataFrame(dados, schema)

# Exibe o DataFrame
display(df_test_null)

# Chamar a função para tratamento de valores nulos no DataFrame
df_trat_null = verificar_e_tratar_valores_nulos(df_test_null)

display(df_trat_null)

id,nome,idade,genero,ativo,data_nascimento,ultima_atualizacao
1,João,30,Masculino,true,1992-05-15,2024-04-29T06:55:34.990153Z
2,Maria,null,Feminino,false,null,2024-04-29T06:55:34.990156Z
3,null,25,null,true,1997-10-20,null


Valores nulos tratados por coluna:
id: 0
nome: 1
idade: 1
genero: 1
ativo: 0
data_nascimento: 1
ultima_atualizacao: 1
Todos os valores nulos foram tratados.


id,nome,idade,genero,ativo,data_nascimento,ultima_atualizacao
1,João,30,Masculino,true,1992-05-15,2024-04-29T06:55:34.990153Z
2,Maria,0,Feminino,false,1900-01-01,2024-04-29T06:55:34.990156Z
3,Não informado,25,Não informado,true,1997-10-20,1900-01-01T00:00:00Z


#### Função para criar arquivo Delta Lake

In [0]:
def criar_arquivo_delta(parquet_file :str, delta_path :str, modo="overwrite") -> None:
    """
    Cria uma tabela Delta a partir de um arquivo Parquet no DBFS do Databricks.

    Parâmetros:
    arquivo_parquet (str): O caminho do arquivo Parquet origem.
    delta_path (str): O caminho onde a tabela Delta será criada.
    modo (str, opcional): O modo de escrita. Pode ser "overwrite" (substitui), "append" (anexa) ou "ignore" (ignora). O padrão é "overwrite".
    """
    try:
        # Ler o arquivo Parquet
        df = spark.read.parquet(parquet_file)

        # Escrever o DataFrame como uma tabela Delta
        df.write.format("delta").mode(modo).save(delta_path)
        print("Arquivo Delta Lake criado com sucesso em:", delta_path)
    except Exception as e:
        print("Erro ao criar arquivo Delta Lake:", str(e))

#### Aplica etapa de Tratamento de Dados (Valores Nulos e Duplicados)

**Premissa**: Realize transformações necessárias, como tratamento de valores nulos, conversões de tipos, etc.

In [0]:
# Parâmetros de leitura de arquivo
path = dbutils.widgets.get('p_path')    #"dbfs:/FileStore/datum/"
layer = dbutils.widgets.get('p_layer')   #"bronze/olistbr-brazilian-ecommerce"
file_name_bronze = dbutils.widgets.get('p_file_name_bronze')   #"olist_customers_dataset"

# Construindo o caminho completo para o arquivo Parquet
file_path_bronze = f"{path}/{layer}/{file_name_bronze}"

# Chamando a função para ler o arquivo Parquet
df = ler_parquet(file_path_bronze)

Leitura do arquivo Parquet bem-sucedida.


In [0]:
# Chamar a função para verificar valores duplicados no DataFrame
df_tratado = verificar_e_remover_duplicatas(df)

# Chamar a função para verificar valores nulos no DataFrame
df_tratado = verificar_e_tratar_valores_nulos(df_tratado)

# display(df_tratado)

NÃO existem linhas duplicadas
Valores nulos tratados por coluna:
customer_id: 0
customer_unique_id: 0
customer_zip_code_prefix: 0
customer_city: 0
customer_state: 0
row_ingestion_timestamp: 0
Todos os valores nulos foram tratados.


#### Opção de grava os dados em parquet ou Delta Lake

In [0]:
# Parâmetros de gravação de arquivo
file_name_silver = dbutils.widgets.get('p_file_name_silver')    #'customers_silver'
destination_path = f'{path}/silver/olistbr-brazilian-ecommerce/{file_name_silver}'
mode_write_silver = dbutils.widgets.get('p_mode_write_silver')   #'overwrite'

# Salva arquivo em formato parquet, na camada bronze
gravar_parquet(df_tratado, destination_path=destination_path, mode_write_bronze=mode_write_silver)

Dados salvos com sucesso no formato Parquet.


In [0]:
# Parâmetros para salvar arquivo delta
table_name = file_name_silver
destination_path_delta = f'dbfs:/FileStore/datum/tables/{table_name}'
modo_write_delta = dbutils.widgets.get('p_modo_write_delta')   #"overwrite"

# Chama função para cria arquivo Delta Lake
# criar_arquivo_delta(parquet_file=destination_path, delta_path=destination_path_delta, modo=modo_write_delta)

#### Cria tabela delta

In [0]:
def save_dataframe_as_table(df :DataFrame, table_name :str) -> str:
    """
    Cria (se necessário) e usa um banco de dados, exclui uma tabela existente,
    e salva um DataFrame como uma nova tabela no Apache Spark.

    Parâmetros:
        df_tratado (DataFrame): O DataFrame que será salvo como tabela.
        table_name (str): O nome da tabela a ser criada/sobreescrita.
    """

    try:

        # Cria o banco de dados 'olist' se ele não existir
        spark.sql('CREATE DATABASE IF NOT EXISTS olist')
        
        # Seleciona o banco de dados 'olist' para uso
        spark.sql('USE olist')
        
        # Exclui a tabela se ela já existir
        spark.sql(f'DROP TABLE IF EXISTS {table_name}')
        
        # Salva o DataFrame como uma tabela, sobrescrevendo se necessário
        df.write.mode('overwrite').saveAsTable(f'{table_name}')

        return f'Tabela {table_name} criada/atualizada com sucesso no banco de dados olist.'

    except Exception as e:
        return f'Ocorreu um erro ao processar a tabela {table_name}: {e}'

In [0]:
# Chama função para cria tabela
save_dataframe_as_table(df_tratado, table_name)

'Tabela customers criada/atualizada com sucesso no banco de dados olist.'